In [56]:
import sys 
import csv
sys.path.append("..")
from src.utils.constantes import DATA_CLEAN_PATH


1- A partir de la información de cada año contenida en el dataset se debe informar, año tras año, el porcentaje de personas mayores a 6 años capaces e incapaces de leer y escribir.
Importante: tomar la información del último trimestre de cada año.

### Tabla individual  
1- ANO4 N (4) Año de relevamiento (4 dígitos)

2- TRIMESTRE N (1) Ventana de observación  
- = 1° trimestre  
- = 2° trimestre  
- = 3° trimestre  
- = 4° trimestre  

9- PONDERA N (6) Ponderación  

12- CH05 date Fecha de nacimiento
(día, mes y año) --> ver como esta dado el dato

13- CH06 N (2) ¿Cuántos años cumplidos tiene?

16- CH09 N (1) ¿Sabe leer y escribir?
1 = Sí
2 = No
3 = Menor de 2 años

________________________________________________________
Columnas usadas: 2 - 1 - 9 - 13 - 16


In [57]:
def separar_por_trimestre(dict_anios): 
    dict_final = {}
    for anio in dict_anios:
        dict_temporal = {}
        
        for filas in dict_anios[anio]:
            trimestre = filas[2]
            if trimestre not in dict_temporal: 
                dict_temporal[trimestre] = []
            dict_temporal[trimestre].append(filas)

        dict_final[anio] = dict_temporal
    return dict_final

def agrupar_por_anio_y_trimestre(filas, col_anio=1):
    grupos = {}
    for fila in filas:
        anio = fila[col_anio]
        if anio not in grupos:
            grupos[anio] = []
        grupos[anio].append(fila)
    grupos_final = separar_por_trimestre(grupos)
    return grupos_final



def porcentaje(valor,total):
    return (valor / total) * 100

def analizar_analfabetismo (dict_ultimos_tri, anio): 
    alfabetos, analfabetos, desconocido, poblacion = 0, 0, 0, 0 
    max_1 = max(dict_ultimos_tri[anio].keys())
    for elem in dict_ultimos_tri[anio][max_1]:
        pondera = int(elem[9])
        poblacion += pondera
        if int(elem[13]) <= 6: 
            continue
        if elem[16] == "1": 
            alfabetos += pondera
        elif elem[16] == "2": 
            analfabetos += pondera
        else:
            desconocido += pondera
    return porcentaje(alfabetos,poblacion), porcentaje(analfabetos,poblacion), porcentaje(desconocido,poblacion)

def activ_1(dict_ultimos_tri):
    for anios in dict_ultimos_tri: 
        por_alfa, por_ana , por_desc = analizar_analfabetismo(dict_ultimos_tri,anios)
        print(f'en {anios} \nel porcentaje de Alfabetismo fue: {por_alfa:.2f}% \nel porcentaje de Analfabetismo fue: {por_ana:.2f}% \nel porcentaje de desconocidos fue: {por_desc:.2f}%')


In [58]:
archivo_clean_path = DATA_CLEAN_PATH / "usu_clean_individual.csv"

f = archivo_clean_path.open("r",encoding="utf-8") 
reader = csv.reader(f,delimiter=";")
header = next(reader)
new_list = []
for elem in reader:
    new_elem = elem[0].split(',')
    new_list.append(new_elem)
    
dict_trimestres = agrupar_por_anio_y_trimestre(new_list)


In [59]:
activ_1(dict_trimestres)

en 2024 
el porcentaje de Alfabetismo fue: 90.81% 
el porcentaje de Analfabetismo fue: 0.84% 
el porcentaje de desconocidos fue: 0.04%
en 2023 
el porcentaje de Alfabetismo fue: 90.42% 
el porcentaje de Analfabetismo fue: 0.84% 
el porcentaje de desconocidos fue: 0.06%


2. A partir de un año y trimestre elegido por el usuario informar el porcentaje de personas no nacidas en Argentina que hayan cursado un nivel universitario o superior.

#### Tabla individual  
1- ANO4 N (4) Año de relevamiento (4 dígitos)

2- TRIMESTRE N (1) Ventana de observación  
- = 1° trimestre  
- = 2° trimestre  
- = 3° trimestre  
- = 4° trimestre  

9- PONDERA N (6) Ponderación  

19- CH12 N (2) ¿Cuál es el nivel más alto que cursa o cursó?
- 1 = Jardín/preescolar
- 2 = Primario
- 3 = EGB
- 4 = Secundario
- 5 = Polimodal
- 6 = Terciario
- 7 = Universitario
- 8 = Posgrado universitario
- 9 = Educación especial (discapacitado)

22- CH15 N (1) ¿Dónde nació?
1. En esta localidad
2. En otra localidad de esta provincia
3. En otra provincia (especificar)
4. En un país limítrofe (especificar: Brasil, Bolivia, Chile, Paraguay, Uruguay)
5. En otro país (especificar)
9. Ns/Nr

26- NIVEL_ED N (1) Nivel educativo
- 1 = Primario incompleto (incluye educación especial)
- 2 = Primario completo
- 3 = Secundario incompleto
- 4 = Secundario completo
- 5 = Superior universitario incompleto
- 6 = Superior universitario completo
- 7 = Sin instrucción
- 9 = Ns/Nr

______________________________
Columnas usadas: 1 - 2 - 9 - 19 - 22

In [60]:
def es_extranjero(valor_ch15):
    return valor_ch15 in ['4', '5']

def es_universitario(valor_ch12):
    return valor_ch12 in ['7', '8']

def activ2 (dict_trimestres,anio_elegido= 0,trimestre_elegido= 0):
    try:
        poblacion, ext_univ = 0, 0
        for elem in dict_trimestres[anio_elegido][trimestre_elegido]:
            pondera = int(elem[9])
            poblacion += pondera
            if not es_extranjero(elem[22]):
                continue 
            if not es_universitario(elem[19]):
                continue 
            ext_univ += pondera
        return porcentaje(ext_univ,poblacion)
    
    except KeyError: 
        print(f'Valores no encontrados en el diccionario')

Se toma el porcentaje de universitarios recibidos sobre la poblacion total, a chequear si es sobre la poblacion de extranjeros

In [72]:
anio_elegido = input("ingrese el año a buscar")
trimestre_elegido = input("ingrese el trimestre a buscar")
porcentaje_ext_univ = activ2(dict_trimestres,anio_elegido,trimestre_elegido)
if porcentaje_ext_univ: 
    print(f'el porcentaje de personas extranjeras universitarias es: {porcentaje_ext_univ:.2f}')

el porcentaje de personas extranjeras universitarias es: 0.98


3. A partir de la información contenida en el dataset informar el año y trimestre donde hubo menor desocupación.

Criterio usado:  
Poblacion en condiciones de trabajar: aproximadamente entre 18 y 65 años varia dependiendo del sexo.  
Población Activa: incluye población ocupada y desempleados (desocupados, personas que no tienen trabajo aunque lo buscan activamente).  
Población Inactiva: gente que no tiene trabajo pero tampoco lo busca.  

Para el ejercicio se cosiderara:  
Población activa = Ocupados + Desocupados  
Tasa de paro o desempleo = (Población desocupada / Población activa) x 100

In [83]:
def calcular_tasa_desempleo (dict_trimestres):
    dict_tasa = {}
    for anio in dict_trimestres:
        if anio not in dict_tasa:
            dict_tasa[anio] = {}
        
        for trimestre in dict_trimestres[anio]:
            
            if trimestre not in dict_tasa[anio]:
                dict_tasa[anio][trimestre] = 0.0
            
            ocupados, desocupados = 0, 0
            for elem in dict_trimestres[anio][trimestre]: 
                pondera = int(elem[9])
                if elem[27] == "1":
                    ocupados += pondera
                elif elem[27] == "2":
                    desocupados += pondera
            
            activos = desocupados + ocupados
            dict_tasa[anio][trimestre] = porcentaje(desocupados,activos)
    return dict_tasa

def calcular_menor_tasa_desempleo(dict_tasa):
    min_tasa = 100
    for anio, trimestres in dict_tasa.items():
        for trimestre, tasa in trimestres.items():
            if tasa < min_tasa:
                min_tasa = tasa
                max_anio = anio
                max_trimestre = trimestre
    return max_anio, max_trimestre, min_tasa

def activ_3(dict_trimestres):
    dict_tasa = calcular_tasa_desempleo(dict_trimestres)
    max_anio, max_trimestre, max_tasa = calcular_menor_tasa_desempleo(dict_tasa)
    print(f"Menor tasa de desempleo: {max_tasa:.2f}% en año {max_anio}, trimestre {max_trimestre}")



In [84]:
activ_3(dict_trimestres)


Menor tasa de desempleo: 5.73% en año 2023, trimestre 3


In [64]:
f.close()